# CH2 실습

## 1. 인덱스와 슬라이스
### 1-2. 자체 시퀀스 생성

In [1]:
class Items:
    def __init__(self, *values):
        self._values = list(values)

    def __len__(self):
        return len(self._values)

    def __getitem__(self, item):
        return self._values.__getitem__(item)

In [4]:
item = Items(1,2,3)
len(item)
item.__getitem__(1)

2

## 2. 컨텍스트 관리자

In [9]:
run = print

def stop_database():
    run("systemctl stop postgresql.service")

def start_database():
    run("systemctl start postgresql.service")

class DBHandler:
    def __enter__(self):
        stop_database()
        return self
    
    def __exit__(self, exc_type, ex_value, ex_traceback):
        start_database()

def db_backup():
    run("pg_dump database")

def main():
    with DBHandler():
        db_backup()
        
if __name__ == "__main__":
    main()

systemctl stop postgresql.service
pg_dump database
systemctl start postgresql.service


In [8]:
import contextlib
@contextlib.contextmanager
def db_handler():
    stop_database()
    yield
    start_database()

with db_handler():
    db_backup()

systemctl stop postgresql.service
pg_dump database
systemctl start postgresql.service


In [12]:
class dbhandler_decorator(contextlib.ContextDecorator):
    def __enter__(self):
        stop_database()

    def __exit__(self, ext_type, ex_value, ex_traceback):
        start_database()

@dbhandler_decorator()
def offline_backup():
    run("pg_dump database")

offline_backup()

systemctl stop postgresql.service
pg_dump database
systemctl start postgresql.service
